<a href="https://colab.research.google.com/github/Mingjie-Shen/Deeep-Learning-Homework-and-Practice/blob/main/Q3_Problem_Sheet_2_(improved).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import h5py
import time

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at MNISTdata.hdf5; to attempt to forcibly remount, call drive.mount("MNISTdata.hdf5", force_remount=True).
Mounted at /content/drive


In [11]:
#load MNIST data
MNIST_data = h5py.File('/content/drive/My Drive/MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] ).reshape((-1,1,28,28))
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] ).reshape((-1,1,28,28))
y_test = np.int32( np.array( MNIST_data['y_test'][:,0]  ) )

MNIST_data.close()

In [12]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5,stride=2)  
        self.conv2 = nn.Conv2d(16, 64, kernel_size=5,stride=1)
        self.fc = nn.Linear(64*8*8, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x,1)
        x = self.fc(x)

        return F.log_softmax(x,dim=1)

In [13]:
model=MyCNN()

In [14]:
optimizer=optim.SGD(model.parameters(),lr=0.1)

In [15]:
batch_size=100
num_epochs=100
L_Y_train=len(y_train)
model.train()
train_loss=[]

In [16]:
time1=time.time()
for epoch in range(num_epochs):
  I_permutation=np.random.permutation(L_Y_train)
  x_train=x_train[I_permutation,:]
  y_train=y_train[I_permutation]
  train_accu=[]
  for i in range(0,L_Y_train,batch_size):
    x_train_batch=torch.FloatTensor(x_train[i:i+batch_size,:])
    y_train_batch=torch.LongTensor(y_train[i:i+batch_size])
    data,target=Variable(x_train_batch),Variable(y_train_batch)

    optimizer.zero_grad()

    output=model(data)
    loss=F.nll_loss(output,target)

    loss.backward()
    train_loss.append(loss.data)

    optimizer.step()
    prediction=output.data.max(1)[1]
    accuracy=(float(prediction.eq(target.data).sum())/float(batch_size))*100
    train_accu.append(accuracy)
  accuracy_epoch=np.mean(train_accu)
  print(epoch,accuracy_epoch)

time2=time.time()
delta_t=time2-time1
print(delta_t)

0 92.23166666666667
1 97.52833333333334
2 98.135
3 98.50666666666666
4 98.72166666666666
5 98.83166666666666
6 98.99166666666666
7 99.075
8 99.23833333333333
9 99.325
10 99.365
11 99.485
12 99.52333333333333
13 99.545
14 99.65
15 99.71
16 99.70833333333333
17 99.76666666666667
18 99.80333333333333
19 99.83333333333333
20 99.825
21 99.88833333333334
22 99.92166666666667
23 99.90166666666667
24 99.89833333333333
25 99.89666666666666
26 99.94
27 99.96
28 99.96166666666667
29 99.95833333333333
30 99.97333333333333
31 99.98
32 99.98
33 99.95666666666666
34 99.99
35 99.98833333333333
36 99.995
37 99.99666666666667
38 99.99333333333334
39 99.99333333333334
40 99.99333333333334
41 100.0
42 99.99833333333333
43 99.99833333333333
44 100.0
45 100.0
46 100.0
47 100.0
48 100.0
49 99.99833333333333
50 100.0
51 100.0
52 100.0
53 100.0
54 100.0
55 100.0
56 100.0
57 100.0
58 100.0
59 100.0
60 100.0
61 100.0
62 100.0
63 100.0
64 100.0
65 100.0
66 100.0
67 100.0
68 100.0
69 100.0
70 100.0
71 100.0
72 100

In [17]:
model.eval()
test_accu=[]
for i in range(0,len(y_test),batch_size):
  x_test_batch=torch.FloatTensor(x_test[i:i+batch_size,:])
  y_test_batch=torch.LongTensor(y_test[i:i+batch_size])
  data,target=Variable(x_test_batch),Variable(y_test_batch)
  optimizer.zero_grad()
  output=model(data)
  loss=F.nll_loss(output,target)
  prediction=output.data.max(1)[1]
  accuracy=(float(prediction.eq(target.data).sum())/float(batch_size)*100)
  test_accu.append(accuracy)

accuracy_test=np.mean(test_accu)
print('The accuracy on the test set is',accuracy_test)

The accuracy on the test set is 98.88
